<a href="https://colab.research.google.com/github/wmangoni/chess_wm_ai/blob/main/chess_will.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-chess
!pip install python-polyglot

# Ative a GPU

Por se tratar de um processamento muito pesado, não esqueça de rodar o código com uma GPU. No Google Colab basta ir em Ambiente de execução >> Alterar tipo de ambiente de execução.

Se tudo der certo, rode o comando abaixo para ver as especificações das GPU

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jan 16 00:35:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import chess
import chess.svg

In [4]:
class MaterialScore:
  def score(self, tabuleiro):
      wp = len(tabuleiro.pieces(chess.PAWN, chess.WHITE))
      bp = len(tabuleiro.pieces(chess.PAWN, chess.BLACK))
      wn = len(tabuleiro.pieces(chess.KNIGHT, chess.WHITE))
      bn = len(tabuleiro.pieces(chess.KNIGHT, chess.BLACK))
      wb = len(tabuleiro.pieces(chess.BISHOP, chess.WHITE))
      bb = len(tabuleiro.pieces(chess.BISHOP, chess.BLACK))
      wr = len(tabuleiro.pieces(chess.ROOK, chess.WHITE))
      br = len(tabuleiro.pieces(chess.ROOK, chess.BLACK))
      wq = len(tabuleiro.pieces(chess.QUEEN, chess.WHITE))
      bq = len(tabuleiro.pieces(chess.QUEEN, chess.BLACK))

      material = 100 * (wp - bp) + 320 * (wn - bn) + 330 * (wb - bb) + 500 * (wr - br) + 900 * (wq - bq)

      return material

In [5]:
pawntable = [
    0, 0, 0, 0, 0, 0, 0, 0,
    5, 10, 10, -20, -20, 10, 10, 5,
    5, -5, -10, 0, 0, -10, -5, 5,
    0, 0, 0, 20, 20, 0, 0, 0,
    5, 5, 10, 25, 25, 10, 5, 5,
    10, 10, 20, 30, 30, 20, 10, 10,
    50, 50, 50, 50, 50, 50, 50, 50,
    0, 0, 0, 0, 0, 0, 0, 0]

knightstable = [
    -50, -40, -30, -30, -30, -30, -40, -50,
    -40, -20, 0, 5, 5, 0, -20, -40,
    -30, 5, 10, 15, 15, 10, 5, -30,
    -30, 0, 15, 20, 20, 15, 0, -30,
    -30, 5, 15, 20, 20, 15, 5, -30,
    -30, 0, 10, 15, 15, 10, 0, -30,
    -40, -20, 0, 0, 0, 0, -20, -40,
    -50, -40, -30, -30, -30, -30, -40, -50]

bishopstable = [
    -20, -10, -10, -10, -10, -10, -10, -20,
    -10, 5, 0, 0, 0, 0, 5, -10,
    -10, 10, 10, 10, 10, 10, 10, -10,
    -10, 0, 10, 10, 10, 10, 0, -10,
    -10, 5, 5, 10, 10, 5, 5, -10,
    -10, 0, 5, 10, 10, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -10, -10, -10, -10, -20]

rookstable = [
    0, 0, 0, 5, 5, 0, 0, 0,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    -5, 0, 0, 0, 0, 0, 0, -5,
    5, 10, 10, 10, 10, 10, 10, 5,
    0, 0, 0, 0, 0, 0, 0, 0]

queenstable = [
    -20, -10, -10, -5, -5, -10, -10, -20,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -10, 5, 5, 5, 5, 5, 0, -10,
    0, 0, 5, 5, 5, 5, 0, -5,
    -5, 0, 5, 5, 5, 5, 0, -5,
    -10, 0, 5, 5, 5, 5, 0, -10,
    -10, 0, 0, 0, 0, 0, 0, -10,
    -20, -10, -10, -5, -5, -10, -10, -20]

kingstable = [
    20, 30, 10, 0, 0, 10, 30, 20,
    20, 20, 0, 0, 0, 0, 20, 20,
    -10, -20, -20, -20, -20, -20, -20, -10,
    -20, -30, -30, -40, -40, -30, -30, -20,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30]

In [6]:
def alphabeta(tabuleiro, alpha, beta, depthleft):
    bestscore = -9999
    if (depthleft == 0):
        return quiesce(tabuleiro, alpha, beta)
    for move in tabuleiro.legal_moves:
        tabuleiro.push(move)
        score = -alphabeta(tabuleiro, -beta, -alpha, depthleft - 1)
        tabuleiro.pop()
        if (score >= beta):
            return score
        if (score > bestscore):
            bestscore = score
        if (score > alpha):
            alpha = score
    return bestscore

In [7]:
def quiesce(tabuleiro, alpha, beta):

    stand_pat = evaluate_board(tabuleiro)

    if (stand_pat >= beta):
        return beta
    if (stand_pat > alpha):
        alpha = stand_pat

    for move in tabuleiro.legal_moves:
        if tabuleiro.is_capture(move):
            tabuleiro.push(move)
            score = -quiesce(tabuleiro, -beta, -alpha)
            tabuleiro.pop()
            if (score >= beta):
                return beta
            if (score > alpha):
                alpha = score
    return alpha

In [8]:
def evaluate_board(tabuleiro):
  if tabuleiro.is_checkmate():
        if tabuleiro.turn:
            return -9999
        else:
            return 9999
  if tabuleiro.is_stalemate():
        return 0
  if tabuleiro.is_insufficient_material():
        return 0

  material = MaterialScore().score(tabuleiro)

  pawnsq = sum([pawntable[i] for i in tabuleiro.pieces(chess.PAWN, chess.WHITE)])
  pawnsq = pawnsq + sum([-pawntable[chess.square_mirror(i)]
                        for i in tabuleiro.pieces(chess.PAWN, chess.BLACK)])
  knightsq = sum([knightstable[i] for i in tabuleiro.pieces(chess.KNIGHT, chess.WHITE)])
  knightsq = knightsq + sum([-knightstable[chess.square_mirror(i)]
                            for i in tabuleiro.pieces(chess.KNIGHT, chess.BLACK)])
  bishopsq = sum([bishopstable[i] for i in tabuleiro.pieces(chess.BISHOP, chess.WHITE)])
  bishopsq = bishopsq + sum([-bishopstable[chess.square_mirror(i)]
                            for i in tabuleiro.pieces(chess.BISHOP, chess.BLACK)])
  rooksq = sum([rookstable[i] for i in tabuleiro.pieces(chess.ROOK, chess.WHITE)])
  rooksq = rooksq + sum([-rookstable[chess.square_mirror(i)]
                        for i in tabuleiro.pieces(chess.ROOK, chess.BLACK)])
  queensq = sum([queenstable[i] for i in tabuleiro.pieces(chess.QUEEN, chess.WHITE)])
  queensq = queensq + sum([-queenstable[chess.square_mirror(i)]
                          for i in tabuleiro.pieces(chess.QUEEN, chess.BLACK)])
  kingsq = sum([kingstable[i] for i in tabuleiro.pieces(chess.KING, chess.WHITE)])
  kingsq = kingsq + sum([-kingstable[chess.square_mirror(i)]
                        for i in tabuleiro.pieces(chess.KING, chess.BLACK)])

  eval = material + pawnsq + knightsq + bishopsq + rooksq + queensq + kingsq
  if tabuleiro.turn:
      return eval
  else:
      return -eval

In [9]:
def selectmove(tabuleiro, depth):
    bestMove = chess.Move.null()
    bestValue = -99999
    alpha = -100000
    beta = 100000

    for move in tabuleiro.legal_moves:
        tabuleiro.push(move)
        boardValue = -alphabeta(tabuleiro, -beta, -alpha, depth - 1)
        print(boardValue)
        if boardValue > bestValue:
            bestValue = boardValue
            bestMove = move
        if boardValue > alpha:
            alpha = boardValue
        tabuleiro.pop()
    return bestMove

In [ ]:
import chess.engine
from polyglot.core import Polyglot
from IPython.display import SVG

def is_legal_move(board, move_uci):
    move = chess.Move.from_uci(move_uci)

    # Verificar se a jogada é válida
    if move in list(board.legal_moves):
        board_copy = board.copy()
        board_copy.push(move)
        return True

    return False

def get_legal_move(move_uci, board):
    if is_legal_move(board, move_uci):
        return chess.Move.from_uci(move_uci)
    else:
        print("ilegal move... play other movement")
        movement = input("Enter your move (in UCI format): ")
        return get_legal_move(movement, board)

def main():

    board = chess.Board()

    count = 0
    movehistory = []

    while not board.is_game_over(claim_draw=True):

        display(board)
        print(movehistory)

        if board.turn == chess.BLACK:
            print("---Black turn---")

            try:
                move = chess.polyglot.MemoryMappedReader("/content/human.bin").weighted_choice(board).move
            except:
                move = selectmove(board, 3)
        else:
            count += 1
            print(f'\n{count}]\n')
            print("---White turn---")
            movement = input("Enter your move (in UCI format): ")
            if movement == "x":
                move = selectmove(board, 3)
            else:
                move = get_legal_move(movement, board)

        movehistory.append(move)
        board.push(move)


    display(board)
    print("Game Over")
    print("Result:", board.result())

if __name__ == "__main__":
    main()
